# S500 Project: Stations Data

Data wrangling of NOAA data and World Bank data using BigQuery associated with the s500-project on ac-99's github. Reading in NOAA & WDI data; cleaning; merging.

The output is the weather_GDP dataframe.

Author: Anubhav Chowdhury, University of Cambridge (MPhil in Economic Research), https://github.com/ac-99

Date: May 2022

Project abstract: This essay exploits year-to-year temperature and precipitation variation to estimate the effects of weather on economic growth using fixed effects methods, a la Dell (2012). This essay hypothesises, based on a large volume of micro-level evidence, that agricultural effects mediate the relationship between weather and economic growth. This problem is analysed using fixed effects regression methods, using dynamic measures for poverty and agriculture, rather than the static measures used by Dell (2021). This analysis is based on a US National Oceanic and Atmospheric Administration (NOAA) dataset with over 90 million observations from over 15,000 weather stations and World Bank Development Indicators between 1992 and 2017.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python 
# For example, here's several helpful packages to load


# Data and Analysis

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import bq_helper package
import bq_helper 
# from bq_helper import BigQueryHelper
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import shutil
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

os.system('pip install linearmodels')
import linearmodels # pip install
from linearmodels import PanelOLS
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
sns.set_palette("magma")

/kaggle/input/fipscountrycodes/country-codes-fips.csv
/kaggle/input/wdi-data-csv-format/WDIData.csv
/kaggle/input/noaa-csv/noaa.csv
/kaggle/input/noaastationslatlon/stations.csv


#### Import country codes
FIPS codes used for later merging.

In [2]:
# Read in data
codes = pd.read_csv('../input/fipscountrycodes/country-codes-fips.csv', encoding='latin-1')
codes=codes[['fips','country']]
codes=codes.rename(columns={'country':'Country','fips':'country'})

## Load in weather data from NOAA GSOD dataset

Steps:
1. Finding stations.
2. Find records from all years.
3. Merge country codes, stations and records to produce overall dataset.
4. (Optional) Just use (previously output) weather records and skip steps 2-3. Saves time, BQ quota.

In [3]:
# Starting and ending years
starting_year = 1992
final_year= 2019
period=final_year-1-starting_year # Infer years

#### 1. Find stations (considering those in 1985, 1995, 2005, 2015 to expedite process) 

Ideally this would involve looking at every year, but limited BigQuery quota requires only using a sample of the stations.

In [4]:
weather_bq = bq_helper.BigQueryHelper(active_project="bigquery-public-data",dataset_name="noaa_gsod")
stations_frames=[] # eventually becomes a list of DFs

for year in [1985,1995,2005,2015]:
    query1 = """SELECT DISTINCT usaf,stn,country, lat, lon
        FROM
          `bigquery-public-data.noaa_gsod.stations` a
        INNER JOIN `bigquery-public-data.noaa_gsod.gsod{}` b ON a.usaf = b.stn
        LIMIT 4000000
            """.format(year)
    
    # For tracking progress
    print(str(year)+" starting...")
    stations = weather_bq.query_to_pandas_safe(query1, max_gb_scanned=1)
    print(str(year)+" downloaded.")
    stations=stations.drop_duplicates()
    print(str(year)+" duplicates dropped.")
    stations['year']=int(year)
    stations_frames.append(stations)
    print(str(year)+" complete.")


Using Kaggle's public dataset BigQuery integration.
1985 starting...


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


1985 downloaded.
1985 duplicates dropped.
1985 complete.
1995 starting...
1995 downloaded.
1995 duplicates dropped.
1995 complete.
2005 starting...
2005 downloaded.
2005 duplicates dropped.
2005 complete.
2015 starting...
2015 downloaded.
2015 duplicates dropped.
2015 complete.


In [5]:
# Merge and drop duplicates from stations df

stations=pd.concat(stations_frames) # Merge
print(stations.shape) # how many stations remain?

stations.to_csv('../stations.csv',index=True) # output file
shutil.copy("../stations.csv", "../working/stations.csv") # Move into working directory
stations=stations.drop('year',axis=1).drop_duplicates() # drop duplicates that exist, based on year duplicates 
print(stations.shape) # how many stations remain?
# stations

(48503, 6)
(19267, 5)


#### 2. Read in weather records (use existing csv as needed)

Use BigQuery and loop through all relevant years. 

In [6]:
weather_frames=[] # ready to merge

# Append a dataframe for each year being considered
for year in range(starting_year,final_year):
        query1 = """SELECT a.usaf,b.stn,a.country,CAST(AVG(b.temp) AS NUMERIC) AS temperature,CAST(AVG(b.prcp) AS NUMERIC) AS precipitation
    FROM
      `bigquery-public-data.noaa_gsod.stations` a
    INNER JOIN `bigquery-public-data.noaa_gsod.gsod{}` b ON a.usaf = b.stn WHERE b.temp!=9999.9 AND b.prcp!=9999.9 
    GROUP BY a.country, a.usaf, b.stn
        """.format(year)
        
        # to keep track of progress
        print(year)
        print(query1)
        
        # to keep track of progress
        a=weather_bq.query_to_pandas_safe(query1, max_gb_scanned=5)
        
        # estimate of query
        print(weather_bq.estimate_query_size(query1))
        
        # Add the correct year to these entries
        a['year']=int(year)
        weather_frames.append(a)

# weather_frames

1992
SELECT a.usaf,b.stn,a.country,CAST(AVG(b.temp) AS NUMERIC) AS temperature,CAST(AVG(b.prcp) AS NUMERIC) AS precipitation
    FROM
      `bigquery-public-data.noaa_gsod.stations` a
    INNER JOIN `bigquery-public-data.noaa_gsod.gsod1992` b ON a.usaf = b.stn WHERE b.temp!=9999.9 AND b.prcp!=9999.9 
    GROUP BY a.country, a.usaf, b.stn
        
0.056892577558755875
1993
SELECT a.usaf,b.stn,a.country,CAST(AVG(b.temp) AS NUMERIC) AS temperature,CAST(AVG(b.prcp) AS NUMERIC) AS precipitation
    FROM
      `bigquery-public-data.noaa_gsod.stations` a
    INNER JOIN `bigquery-public-data.noaa_gsod.gsod1993` b ON a.usaf = b.stn WHERE b.temp!=9999.9 AND b.prcp!=9999.9 
    GROUP BY a.country, a.usaf, b.stn
        
0.05765676125884056
1994
SELECT a.usaf,b.stn,a.country,CAST(AVG(b.temp) AS NUMERIC) AS temperature,CAST(AVG(b.prcp) AS NUMERIC) AS precipitation
    FROM
      `bigquery-public-data.noaa_gsod.stations` a
    INNER JOIN `bigquery-public-data.noaa_gsod.gsod1994` b ON a.usaf = b.stn 

#### 3. Concatenate and merge

Merge the codes, station and weather records data. Grouping by country-wide annual average. 

Given more time, several other approaches could have been used: using population-weighting, subnational regions etc. See essay for details.

In [7]:
# Concatenate the past weather frames
noaa = pd.concat(weather_frames)

# Merge on stations and country codes
noaa=pd.merge(noaa,stations)
noaa=pd.merge(codes,noaa)
noaa.drop(axis=1,columns=['country','usaf','stn','lat', 'lon'],inplace=True) # remove repeat columns

# Get country-wide averages. A more robust analysis would use population weighting schemes, subnational regions etc, (eg Zhao et al 2018)
noaa=noaa.groupby(by=['Country','year']).mean()
# os.chdir(r'/kaggle/working')
noaa.to_csv('../noaa.csv',index=True) # output file
shutil.copy("../noaa.csv", "../working/noaa.csv")

'../working/noaa.csv'

#### 4. (Optional) Now use read-in data so that BQ quota isn't used up each time

This allows the above to be commented out, saving runtime and BQ quota.

In [8]:
noaa_weather = pd.read_csv('../input/noaa-csv/noaa.csv') # read in the previous output (the code for which need only be run once)

print(noaa_weather.shape) # how many data points 
print("Sanity check:")
print(noaa_weather.head())

noaa_weather['temperature']=(noaa_weather['temperature'] - 32) *(5/9) # convert to celcius
print("Sanity check of conversion:")
print(noaa_weather.head()) # Sanity check

(6399, 4)
Sanity check:
       Country  year  temperature  precipitation
0  Afghanistan  1992    54.255075      17.930352
1  Afghanistan  1993    54.080898      36.498651
2  Afghanistan  1994    64.206107      25.722498
3  Afghanistan  1995    60.988315      21.461210
4  Afghanistan  1996    60.586448      27.588678
Sanity check of conversion:
       Country  year  temperature  precipitation
0  Afghanistan  1992    12.363930      17.930352
1  Afghanistan  1993    12.267165      36.498651
2  Afghanistan  1994    17.892281      25.722498
3  Afghanistan  1995    16.104619      21.461210
4  Afghanistan  1996    15.881360      27.588678


### Read in WDI Data

Reading in the World Bank Development Indicators data.

#### Pull out relevant data on each country 

Several factors are included:
* Agricultural sector size (Agriculture, forestry, and fishing, value added (% of GDP))
* GDP, PPP (constant 2011 international $)
* Total greenhouse gas emissions (kt of CO2 equivalent)
* Average precipitation in depth (mm per year)
* GDP growth (annual %)

In [9]:
# Pull out World Bank Development Indicator data
WDI = pd.read_csv('../input/wdi-data-csv-format/WDIData.csv')

# Ideally, the below replacements should use a more robust system like fuzzy matching. These line up with the definitions in the codes.
WDI['Country Name'].replace(to_replace="United States",value="United States of America",inplace=True) # fix for later
WDI['Country Name'].replace(to_replace='Korea, Dem. People’s Rep.',value="Korea, North",inplace=True) # fix for later
WDI['Country Name'].replace(to_replace="Korea, Rep.",value="Korea, South",inplace=True) # fix for later

In [10]:
# Find codes for GDP, Agriculture, Greenhouse Gas Emissions, Average Precipitation

gdp_code=WDI['Indicator Code'][WDI['Indicator Name']=="GDP, PPP (constant 2011 international $)"].unique()[0]
agriculture_code=WDI['Indicator Code'][WDI['Indicator Name']=="Agriculture, forestry, and fishing, value added (% of GDP)"].unique()[0]
c02_code=WDI['Indicator Code'][WDI['Indicator Name']=="Total greenhouse gas emissions (kt of CO2 equivalent)"].unique()[0]
rain_code=WDI['Indicator Code'][WDI['Indicator Name']=="Average precipitation in depth (mm per year)"].unique()[0]
popn_code="SP.POP.TOTL"
growth_code=WDI['Indicator Code'][WDI['Indicator Name']=='GDP growth (annual %)'].unique()[0]

print(gdp_code,agriculture_code,c02_code,rain_code,growth_code) # Sanity check

# Found using: WDI['Indicator Name'][WDI['Indicator Name'].str.contains("GDP")].unique()

NY.GDP.MKTP.PP.KD NV.AGR.TOTL.ZS EN.ATM.GHGT.KT.CE AG.LND.PRCP.MM NY.GDP.MKTP.KD.ZG


#### Get into long format

In [11]:
# Choose WDI data which are 
WDI=WDI[(WDI['Indicator Code']==gdp_code )|(WDI['Indicator Code']==agriculture_code)|
        (WDI['Indicator Code']==popn_code)|(WDI['Indicator Code']==growth_code)] # (WDI['Indicator Code']==rain_code)|(WDI['Indicator Code']==c02_code)|

# Years included in the dataset
WDI_years=list(range(1960,2018))
WDI_years=[str(int) for int in WDI_years] # string type

# Reformat for convenience
WDI=WDI.melt(id_vars=['Indicator Name','Country Name'], value_vars=WDI_years)
WDI=WDI.pivot(index=['Country Name', 'variable'],columns='Indicator Name', values='value')

#### Alter some names for later convenience

In [12]:
# Ensure all names are correct for later merging
WDI=WDI.reset_index()
WDI=WDI.rename(columns={"Country Name": "Country", "variable": "year"})
WDI['Country'][WDI['Country']=='United States of America'] = 'United States'
WDI['year']=WDI['year'].astype('int')
WDI.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Indicator Name,Country,year,"Agriculture, forestry, and fishing, value added (% of GDP)",GDP growth (annual %),"GDP, PPP (constant 2011 international $)","Population, total"
0,Afghanistan,1960,NaN,NaN,NaN,8996351.0
1,Afghanistan,1961,NaN,NaN,NaN,9166764.0
2,Afghanistan,1962,NaN,NaN,NaN,9345868.0
3,Afghanistan,1963,NaN,NaN,NaN,9533954.0
4,Afghanistan,1964,NaN,NaN,NaN,9731361.0


## Final output: making the weather_GDP dataset for regression

The final output is a long-formal dataframe, setup for time and entity effects. This allows the linearmodels package to be used immediately.

In [13]:
# Merge data
weather_GDP=pd.merge(noaa_weather,WDI)

# Rename for usability
weather_GDP=weather_GDP.rename(columns={"Agriculture, forestry, and fishing, value added (% of GDP)": "agricultural", 
                            "temperature": "avg_temp",
                           "precipitation":"avg_precip",
                           "Population, total":"population",
                           "Total greenhouse gas emissions (kt of CO2 equivalent)":"c02",
                                       "GDP, PPP (constant 2011 international $)":"gdp",
                                       "Country":"country",
                                       "Year":"year",
                                       "GDP growth (annual %)":"growth"}
                              )

#### Defining median values

Median values are defined based on the distribution of values from a given year. This is a departure from Dell et al (2012), but better accounts for changes in wealth levels, particularly important for emerging economies (otherwise, this could yield biased estimates).

In [14]:
# Defining median GDP
median_GDP=weather_GDP.groupby(['year']).median()['gdp'].reset_index()
median_GDP=median_GDP.rename(columns={'gdp':'year_median_gdp'})
weather_GDP=pd.merge(weather_GDP,median_GDP,on='year')


# Defining median temp
median_temp=weather_GDP.groupby(['year']).median()['avg_temp'].reset_index()
median_temp=median_temp.rename(columns={'avg_temp':'year_median_temp'})
weather_GDP=pd.merge(weather_GDP,median_temp,on='year')

# Define dummy variables
weather_GDP['poor']=weather_GDP['gdp']<weather_GDP['year_median_gdp']
weather_GDP['hot']=weather_GDP['avg_temp']>weather_GDP['year_median_temp']

# Defining median population -- for plotting
median_pop=weather_GDP.groupby('country').mean()['population'].describe()[5]
weather_GDP['large']=weather_GDP['population']>median_pop

In [15]:
# Examining the number of countries around at the time
weather_GDP.describe()

,year,avg_temp,avg_precip,agricultural,growth,gdp,population,year_median_gdp,year_median_temp
count,4719.000000,4719.000000,4719.000000,4060.000000,4334.000000,4.196000e+03,4.713000e+03,4.719000e+03,4719.000000
mean,2004.611146,20.029650,7.200171,13.264584,3.719830,4.395716e+11,3.297886e+07,4.963723e+10,22.622467
std,7.494317,7.767268,10.033396,12.738584,6.270081,1.530605e+12,1.311404e+08,1.443549e+10,0.528670
min,1992.000000,-8.983333,0.000000,0.026471,-62.075920,2.256761e+07,9.109000e+03,3.128328e+10,21.710681
25%,1998.000000,13.492492,0.764517,2.985655,1.584723,1.204349e+10,1.152309e+06,3.685132e+10,22.296407
50%,2005.000000,22.548929,3.399862,8.694559,3.753243,4.790246e+10,6.192560e+06,4.683447e+10,22.523727
75%,2011.000000,26.559498,9.078224,20.598560,6.013680,2.635919e+11,1.990898e+07,5.642369e+10,22.864531
max,2017.000000,42.000000,99.990000,66.032729,149.972963,2.122392e+13,1.386395e+09,8.056325e+10,23.830523


#### Output data

In [16]:
weather_GDP.to_csv('../weather_GDP.csv',index=True) # 
shutil.copy("../weather_GDP.csv", "../working/weather_GDP.csv") 

'../working/weather_GDP.csv'